# Algoritmo MCTS

Precisamos de importar os seguintes modulos: random, time, mathe e deepcopy, bem como a classe ConnectFour do ficheiro connect4.py.

In [ ]:
import random
import time
import math
from copy import deepcopy
import math

from connect4 import ConnectFour

Começamos por criar uma constante, que corresponde a raiz quadrada de 2 (vai ser usada na fórmula UCB).
De seguida criamos a classe Node, onde cada nó vai corresponder a uma possivel jogada no jogo.

A função init é o construtor da classe que contêm os seguintes atributos:
- move -> que corresponde ao nº da coluna onde o jogador vai jogar
- parent -> que é o nó pai
- N -> que vai ser usada para contar o nº de vezes que o nó vai ser explorado
- Q -> o valor estimado da qualidade deste nó
- children -> que é um dicionário vazio onde vão ser armazenados os nós filhos do nó atual.
- outcome -> o resultado do jogo neste nó


In [ ]:
EXPLORATION = math.sqrt(2)

class Node:
    def __init__(self, move, parent, state=ConnectFour()):
        self.move = move
        self.parent = parent
        self.N = 0
        self.Q = 0
        self.children = {}
        self.outcome = state.PLAYER_X

De seguida, adicionamos o método add_children à classe Node, que adiciona nós filhos a um nó especifico da árvore de pesquisa do MCTS. O seu papel é atualizar a estrutura da árvore de pesquisa à medida que mais simulações são executadas.

In [ ]:
    def add_children(self, children: dict) -> None:
        for child in children:
            self.children[child.move] = child

Criamos também o método value que é responsável por calcular o valor de um nó na árvore de pesquisa do MCTS.

Começamos por verificar se o nó já foi visitado alguma vez, se não tiver sido significa que ainda não foi explorado e então o seu valor é definido conforme o parâmetro explore. Se explore for zero, o valor retornado é zero, indicando que o nó deve ser totalmente explorado. Caso contrário, o valor retornado é infinito, indicando que o nó deve ser explorado imediantamente.

Se o nó foi visitado pelo menos uma vez, a função calcula o valor do nó usando a fórmula UCB que equilibra a exploração e a explotação, tendo em consideração a qualidate estimada do nó e a quantidade de vezes que o nó pai foi visitado em relação ao nº de vezes do nó atual.

In [ ]:
    def value(self, explore: float = EXPLORATION):
        if self.N == 0:
            return 0 if explore == 0 else float('inf')
        else:
            return self.Q / self.N + explore * math.sqrt(math.log(self.parent.N) / self.N)

Criamos uma classe MCTS que representa a implementaçã do algoritmo Monte Carlo Tree Search.

A função init é o construtor da classe que contêm os seguintes atributos:
- root_state -> que cria uma cópia do estado do jogo
- root -> que inicializa o nó da raiz da árvore de pesquisa como um novo objeto da classe Node.
- run_time -> que armazena o tempo total de execução do algoritmo
- node_count -> que conta o nº total de nós criados na árvore de pesquisa
- num_rollouts -> que armazena o nº total de simulações executadas durante o processo de busca


In [ ]:
class MCTS:
    def __init__(self, state=ConnectFour()):
        self.root_state = deepcopy(state)
        self.root = Node(None, None)
        self.run_time = 0
        self.node_count = 0
        self.num_rollouts = 0

Posterirormente, criamos a função select_node que seleciona um nó na árvore de pesquisa durante a fase de seleção do algoritmo.

Inicializamos a variável node como o nó raiz da árvore de pesquisa e state como uma cópia do estado inicial do jogo.

Enquanto o nó atual tem filhos, seleciona-se os filhos com maior valor, utilizando a função max para comparar os valores dos nós filhos. Se existir mais de um filho com o mesmo valor máximo, um deles é escolhido aleatoriamente. O nó escolhido é atualizado como o novo nó atual e o movimento associado é aplicado ao estado do jogo.

Se o nó selecionado não foi visitado ainda, o método retorna esse nó e o estado do jogo associado a este.

Se o loop terminar, porque o nó atual não possui filhos é feita uma chamada para a função expand para expandir a árvore de busca a partir de este nó. Se esta for bem sucedida, um filho é escolhido aleatoriamente e atualizado como o novo nó atual e o movimento associado é aplicado ao estado do jogo.

In [ ]:
    def select_node(self) -> tuple:
        node = self.root
        state = deepcopy(self.root_state)

        while len(node.children) != 0:
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            max_nodes = [n for n in children if n.value() == max_value]

            node = random.choice(max_nodes)
            state.move(node.move)

            if node.N == 0:
                return node, state

        if self.expand(node, state):
            node = random.choice(list(node.children.values()))
            state.move(node.move)

        return node, state

A função expand expande a árvore de pesquisa a partir de um nó especifico.

Primeiro vê se jogo já terminou, se já tiver terminado não é possivel expandir mais a árvore de busca a partir deste nó, então o método retorna Falso.
Depois, obtêm-se uma lista de todas as jogadas possiveis e cria-se uma lista de nós filhos para expandir a árvore. Adiciona-se os nós filhos á arvore de pesquisa chamando o método add_children do nó pai.

In [ ]:
    def expand(self, parent: Node, state: ConnectFour) -> bool:
        if state.game_over():
            return False

        children = [Node(move, parent) for move in state.get_legal_moves()]
        parent.add_children(children)

        return True

Criamos também a função roll_out que realiza um simulação a partir de um determinado estado do jogo até ao fim deste.

Enquanto o jogo não acabar, um movimento é escolhido aleatoriamente entre as jogadas possiveis e é realizado aplicando-o ao estado do jogo.

In [ ]:
    def roll_out(self, state: ConnectFour) -> int:
        while not state.game_over():
            state.move(random.choice(state.get_legal_moves()))

        return state.get_outcome()

O método back_propagate atualiza os valores de N e Q de todos os nós na árvore de pesquisa, retrocedendo a partir de um nó especifico até a raiz.

Começa por definir um reward. Se o resultado for igual ao do último jogador, o reward é 0, indicando que o jogador ganhou. Caso contrário é 1, indicando que o jogador perdeu.

Inicia-se um loop, onde o nº de visitas(N) é incrementado em 1, indicando que foi visitado mais de uma vez e o reward é adicionado ao valor Q do nó.
Move-se para o nó pai do nó atual na árvore de pesquisa.
Por fim, atualiza o reward para o próximo nó com base no resultado da simulação. Se o resultado for um empate o reward é definica como 0. Caso contrário é ajustada para 1 menos o reward atual.

In [ ]:
    def back_propagate(self, node: Node, turn: int, outcome: int, state: ConnectFour) -> None:
        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.Q += reward
            node = node.parent
            if outcome == state.resultado['empate']:
                reward = 0
            else:
                reward = 1 - reward

Criamos a função search que executa a etapa principal do algoritmo.

Começamos por registar o tempo de início da pesquisa.

Inicializamos o contador de simulações para acompanhar o  nº total de simulações realizadas durante a pesquisa.

De seguida, iniciamos um loop que continua enquanto o tempo decorrido desde o inicio da pesquisa for menor que o limite de tempo especificado.

Seleciona um nó na árvore de pesquisa usando o método select_node e depois realiza uma simulação a partir do estado atual do jogo usando o método roll_out.

Atualizam-se os valores de N e Q de todos os nós visitados durante a simulação.

Incrementa-se o contador de simulações após cada simulação realizada.

Calcula-se o tempo total de exceção da busca subraindo o tempo de inicio do tempo atual.

Por fim, atualiza os atributos run_time e num_rollouts da instância da classe com os valores calculados.

In [ ]:
    def search(self, time_limit: int):
        start_time = time.process_time()

        num_rollouts = 0
        while time.process_time() - start_time < time_limit:
            node, state = self.select_node()
            outcome = self.roll_out(state)
            self.back_propagate(node, state.to_play, outcome, state)
            num_rollouts += 1

        run_time = time.process_time() - start_time
        self.run_time = run_time
        self.num_rollouts = num_rollouts

A função best_move sleciona o melhor movimento disponível.

Começa por verificar se o jogo terminou, e se isto acontecer não há movimentos disponiveis, logo retorna -1.

Encontra o valor máximo de visitas (N) entre todos os nós filhos e obtêm todos os nós filhos que têm o valor máximo de visitas.

Escolhe aleatoriamente um dos nós filhos que tem o valor máximo de visitas e retorna a jogada associada ao melhor nó filho selecionado.

In [ ]:
    def best_move(self):
        if self.root_state.game_over():
            return -1

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        best_child = random.choice(max_nodes)

        return best_child.move

O método move atualiza o estado do jogo e o nó raiz da árvore de pesquisa após ser feito um determinado movimento.

Primeiro verifica se o movimento está entre os filhos do nó raiz. Se estiver significa que esse movimento foi explorado durante a busca MCTS e há um nó filho correspondente na árvore de busca.
Atualiza o estado do jogo para refletir o movimento realizado e atualiza o nó da raiz para ser o nó correspondente ao movimento realizado na árvore de busca. Retorna, indicando que a função terminou de ser executada após atualizar o estado do jogo e o nó raiz.

Se o movimento não estiver entre os filhos do nó raiz, significa que o movimento não foi explorado durante a pesquisa MCTS e a árvore de pesquisa precisa de ser reiniciada. Assim, o método atualiza o estado do jogo chamando o método move do objeto self.root_state com o movimento e, em seguida, redefine o nó raiz para ser um novo nó com None como movimento e pai.

In [ ]:
    def move(self, move):
        if move in self.root.children:
            self.root_state.move(move)
            self.root = self.root.children[move]
            return

        self.root_state.move(move)
        self.root = Node(None, None)

Por fim, criamos o método statistics que retorna estatísticas sobre a última execução de pesquisa MCTS. Este retorna um tuplo que contém o nº total de simulações e o tempo de execução da pesquisa.

In [ ]:
    def statistics(self) -> tuple:
        return self.num_rollouts, self.run_time